In [ ]:
# Carga de librerías necesarias para el funcionamiento
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.integrate import quad

## 1. Aproximación de la integral $\int_{0}^{\pi} e^{-x^2/2} dx$
Implementar Newton-Cotes (Trapecio y Simpson) y registrar el error para $n=10, 20, 50, 100$.

In [ ]:
# --- DEFINICIÓN DE MÉTODOS ---
def trapecio_compuesto(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    return (h / 2) * (y[0] + 2 * np.sum(y[1:-1]) + y[-1])

def simpson13_compuesto(f, a, b, n):
    if n % 2 != 0: n += 1
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    return (h / 3) * (y[0] + 4 * np.sum(y[1:-1:2]) + 2 * np.sum(y[2:-2:2]) + y[-1])

# --- RESOLUCIÓN ---
f1 = lambda x: np.exp(-x**2 / 2)
a1, b1 = 0, np.pi
exact_val1, _ = quad(f1, a1, b1)

print(f"{'n':<5} | {'Error Trapecio':<18} | {'Error Simpson':<18}")
print("-" * 50)
for n in [10, 20, 50, 100]:
    res_t = trapecio_compuesto(f1, a1, b1, n)
    res_s = simpson13_compuesto(f1, a1, b1, n)
    print(f"{n:<5} | {abs(res_t - exact_val1):.5e}        | {abs(res_s - exact_val1):.5e}")

## 2. Comparación de tiempos de ejecución
Integral $\int_{0}^{1} \frac{\ln(1+x)}{x^2+1} dx$.

In [ ]:
f2 = lambda x: np.log(1 + x) / (x**2 + 1)
a2, b2 = 0, 1
n_bench = 5000
loops = 50

t0 = time.perf_counter()
for _ in range(loops): trapecio_compuesto(f2, a2, b2, n_bench)
t_trap = (time.perf_counter() - t0) / loops

t0 = time.perf_counter()
for _ in range(loops): simpson13_compuesto(f2, a2, b2, n_bench)
t_simp = (time.perf_counter() - t0) / loops

print(f"Tiempo promedio Trapecio: {t_trap:.6f} s")
print(f"Tiempo promedio Simpson:  {t_simp:.6f} s")

## 3. Resolución de EDO: $y' + y = \sen(x), y(0)=0$
Comparación gráfica de Euler Explícito e Implícito vs Solución Exacta.

In [ ]:
# --- DEFINICIÓN DE MÉTODOS EULER ---
def euler_explicito(x0, y0, h, n_steps):
    x = np.linspace(x0, x0 + n_steps*h, n_steps + 1)
    y = np.zeros(n_steps + 1)
    y[0] = y0
    for i in range(n_steps):
        y[i+1] = y[i] + h * (np.sin(x[i]) - y[i])
    return x, y

def euler_implicito(x0, y0, h, n_steps):
    x = np.linspace(x0, x0 + n_steps*h, n_steps + 1)
    y = np.zeros(n_steps + 1)
    y[0] = y0
    for i in range(n_steps):
        y[i+1] = (y[i] + h * np.sin(x[i+1])) / (1 + h)
    return x, y

def exacta(x):
    return 0.5 * (np.sin(x) - np.cos(x) + np.exp(-x))

# --- GRÁFICA ---
x_start, x_end = 0, 1
y_init = 0
N_values = [10, 50, 80]

# Configuración manual de tamaño para evitar errores de layout
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)
fig.suptitle("Comparación Euler Explícito vs Implícito")

for idx, n in enumerate(N_values):
    h = (x_end - x_start) / n
    x_ex, y_ex = euler_explicito(x_start, y_init, h, n)
    x_im, y_im = euler_implicito(x_start, y_init, h, n)
    
    x_smooth = np.linspace(x_start, x_end, 200)
    y_true = exacta(x_smooth)
    
    ax = axes[idx]
    ax.plot(x_smooth, y_true, 'k-', alpha=0.5, label='Exacta')
    ax.plot(x_ex, y_ex, 'r--.', label='Explícito')
    ax.plot(x_im, y_im, 'b--.', alpha=0.6, label='Implícito')
    
    ax.set_title(f"N={n} (h={h:.2f})")
    ax.set_xlabel("x")
    ax.grid(True)
    if idx == 0:
        ax.set_ylabel("y(x)")
        ax.legend()

plt.subplots_adjust(top=0.85, wspace=0.3)
plt.show()